In [3]:
import torch

In [7]:
%load_ext autoreload
%autoreload 2

****
##### Testing mha 

In [91]:
from mha import muliheaded_attention

In [92]:
testing_mha=muliheaded_attention(4,2)

In [93]:
## Checking the module list output. We based on the number of heads we pass, we will have that many linear layers.
testing_mha.q_mapping

ModuleList(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [95]:
# A random tensor representing 1 image, 5 patches and embedding size of each patch to be 4
testing_tensor=torch.randn(2,5,4)
print(testing_tensor)
testing_mha(testing_tensor).shape

tensor([[[-0.2625,  0.2740, -0.2138,  0.6563],
         [ 0.2145, -0.5587,  0.0124,  0.6617],
         [ 1.4776,  0.0247,  0.1867,  0.4367],
         [-0.4658, -1.9022,  1.7393,  1.5262],
         [-0.6221, -0.2175,  0.5872, -0.6206]],

        [[ 0.1950,  0.6210, -1.5737,  1.7337],
         [-0.9654,  0.8480,  1.0328, -0.8368],
         [-1.5797, -1.4777,  0.1693,  0.0318],
         [ 0.4615,  1.2474,  0.1352,  0.2735],
         [-0.1720,  1.0442,  0.9636,  1.7562]]])
torch.Size([5, 4])
tensor([[-0.1146, -0.6354],
        [-0.0770, -0.6512],
        [ 0.1110, -0.7642],
        [-0.1764, -0.5866],
        [-0.1533, -0.6099]], grad_fn=<MmBackward0>)
tensor([[-1.2227, -0.0050],
        [-1.2072, -0.0021],
        [-1.1608,  0.0028],
        [-1.2190,  0.0178],
        [-1.0228,  0.0423]], grad_fn=<MmBackward0>)
torch.Size([5, 4])
tensor([[-0.4493, -0.4046],
        [-0.7134, -0.2643],
        [-0.5432, -0.3444],
        [-0.4751, -0.3930],
        [-0.5797, -0.3361]], grad_fn=<MmBackward

torch.Size([2, 5, 4])

### Output says that 2 heads will perform calculations on 5 patches where embedding size is 4. Since we have 2 heads, the embedding will be broken down into size of 2 and 2 for each head.

***

#### Testing out patchify function

In [11]:
from vit_model import patchify

# Simulating an input where we have 3 images, each has 3 color channels. Each image has shape 4,4
testing_patchfy=torch.randn(3,3,4,4)

# The output in this case implies, each image has been broken down into 4 patches. Each patch has embedding dim=12
patchify(testing_patchfy,2)

torch.Size([3, 4, 12])
